### Examples for working with seqtables ###

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, '../../')
import pandas as pd
import numpy as np
from __future__ import absolute_import
from seqtables import seqtables
from seqtables.utils.insilica_sequences import generate_sequence
from seqtables.io import create_scratch_data

C:\Anaconda3\lib\importlib\_bootstrap.py:321: FutureWarning:

The pandas.tslib module is deprecated and will be removed in a future version.



In [3]:
from sys import getsizeof
import time

In [4]:
def utf8len(s):
    return len(s.encode('utf-8'))

** Creating seqtables **

The following shows how to create seqtables via fastq

*Note: Simlarly to read_fastq function, there is a read_sam function that loads results from a samfile into seqtable(refer to doc)*

In [ ]:
# read a fastq file (using biopython) into as a seqtable (not as useful usually as using a prealigned sam file)
t1 = time.time()
st = read_sequences.read_fastq('../tests/files/r1.fastq', use_pandas=False)
t2 = time.time()
print(t2-t1)

In [ ]:
# read the same file except using pd.read_csv rather than biopython

t1 = time.time()
st = read_sequences.read_fastq('../tests/files/r1.fastq', use_pandas=True)
t2 = time.time()
print(t2-t1)

We also can create a seqtable by passing in a list of sequences and or qualities

In [ ]:
sttmp = seq_tables.seqtable(
    ['AACCAAAGGA', 'AAAAAAAA', 'AAATTGAAAGA'], 
    qualitydata=['AAGCA?AGGA', '/ABAAAEA', 'FAA?T[GAAGA' ],
)
sttmp

Finally for test purposes we can simply create some fake data to work with seqtables

*Fake data will be site-saturated at positions 5, 10, and 15*

In [5]:
seqs, quals, wt_seq = create_scratch_data(10000, 300, ss_pos=[5, 10, 15])

In [6]:
# reference sequence we made
wt_seq

'TAACTTAGACGGCGTGCGGTGGACGCATACGTGAAAAGTCTTGGTGATGCGATCGGAAGTCCCAACAGTCAAGGAATCGAACTGATTTGGCTGCACAGGTGGCCTCTGACCTTCCGCCCCGCGGCTGCGTCCAAAGATGTCAGTGCTTGCGAAAACGGAACCGTCCAAGCCCATCACCCAAGAAAAGGGGAACTAAACCCCCTCCTCAAATTGCCCACGCCTCTCTCCCACCCGTCGAGGACGGTACGGACCCTACGATTACTCAGGACTAGGCGTGGTTGCCCCGGGGTAGAAGGTATA'

In [ ]:
tmp = np.array(list(seqs), dtype='S').view('S1').reshape(len(seqs), -1)

In [ ]:
tmp[:20]

** Working with quality scores **

**Plotting data**

Using plotly, we can plot how the quality scores are distributed across the sequence data. Similarly to fastqc, we can define bins for grouping together regions in the sequence and generate box plots of the median quality scores in each bin

In [ ]:
t1 = time.time()
qual_dists_plots, qual_dist_df, qual_dist_stats = st.get_quality_dist()
t2 = time.time()
t2-t1

The second element returned by the function get_quality_dist returns an aggregated dataframe where each row represents a quantile defined by the user and each column represents the bins used to merge the data (i.e. 255-259 means that quality scores at base positions 255 to 259 were merged together before getting the descriptive statistics)

In [ ]:
qual_dist_df

The last element returned by the function get_quality_dist returns an aggregated dataframe where each row represents a descriptive statistic and each column represents the bins used to merge the data (i.e. 255-259 means that quality scores at base positions 255 to 259 were merged together before getting the descriptive statistics)

The statistics it returns are:
1. min value in a bin
2. max value in a bin
3. mean quality score of bin
4. median quality score of bin

In [ ]:
qual_dist_stats

In [ ]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode()

The first element returned by get_quality_dist is a list of box plot information that can be used by plotly to generate interactive plots

In [ ]:
qual_dists_plots

In [ ]:
iplot(qual_dists_plots)

** Removing low quality bases **

Seqtables can keep track of both base and quality score at a given position

In [ ]:
#bases
st.seq_table.values.view('S1')

In [ ]:
# corresponding qualities
st.qual_table.values

Because each index position within the table are connected, we can easily convert low quality bases to N by matching the two tables

Lets convert all bases lower than 20 to N

In [ ]:
st.convert_low_bases_to_null(q=20, replace_with='N', inplace=True)

Lets see how many low quality bases we generated at each position

In [ ]:
total_n_by_pos = (st.seq_table == ord('N')).sum(axis=0)
total_n_by_pos

In [ ]:
iplot([go.Scatter(x=st.seq_table.columns, y=total_n_by_pos.values)])

*Note we could have gotten same answer if we just looked at the qual_table*

In [ ]:
low_qual_by_pos = (st.qual_table < 20).sum(axis=0)
iplot([go.Scatter(x=st.seq_table.columns, y=low_qual_by_pos.values)])

**Sequence logos **

We can also generate sequence logos or the distribution of sequences at each position

In this example we can see the positions used for a site-saturation library at base positions, 5, 10, and 15

In [ ]:
iplot(st.seq_logo()[0])

Most of the common metrics used to look at diversity at a given position should be available

In [ ]:
st.get_seq_dist(method='counts')

In [ ]:
# using frequency
st.get_seq_dist(method='freq', ignore_characters='N')

In [ ]:
# using entropy/bits
st.get_seq_dist(method='bits', ignore_characters='N')

** Analyzing sequences to a reference **

Seqtables was created mainly for situations where one wanted to observe how variants within a set differed from a reference. For example, in protein engineering experiments where we create a set of variants from a scaffold or in immunological experiments where one wants to see how the framework or cdr regions evolved from germline

The following will show some really simple examples for looking at select mutations that perform better than using functions such as distance.hamming

**Mutation profile**

First we can check whether there are any biases in the types of mutations we observed (i.e. is there a bias for A->C mutations?)

In [ ]:
st.mutation_profile(wt_seq)

In [ ]:
wt_seq

We can also aggregate the above mutation profile even more and only look for transition or transversion mutations

In [ ]:
#returns => (ratio of TS/TV freq, ts freq, tv freq)
st.mutation_TS_TV_profile(wt_seq)

** Where mutations occur **

In addition we can plot where mutations occur with regard to the reference sequence


In [ ]:
diff_table = st.compare_to_reference(wt_seq, flip=True, ignore_characters=['N'])
iplot(go.Figure(
        data=[go.Scatter(x=st.seq_table.columns, y=diff_table.sum(axis=0))],
        layout=go.Layout(
            xaxis=dict(title='base position'),
            yaxis=dict(title='Mutation counts')
        )
))

** Hamming distance **

In the test data above we created a library within an error prone rate of 0.01 and 3 positions of site saturation. So we can look at the distribution of hamming distance for each sequence

*Another added benefit of using seqtables rather than distance.hamming is we can explicitly define characters we do not want to include when calculating percent of mutations or characters we want to force to always be equal to the wildtype sequence*


In [ ]:
mut_freq_per_seq = st.hamming_distance(wt_seq, ignore_characters=['N'], normalized=True)

In [ ]:
iplot(go.Figure(
        data=[go.Histogram(x=mut_freq_per_seq.values)],
    layout=go.Layout(title='Histogram of mutated bases per sequence')))

**Sequence substrings**

Often we are interested in only a specific region within a sequence. This is especially true when considering site-saturated libraries. In our example above we generated a library where we saturated 3 positions with ACTorG (N). 


We can slice the data and only extract those regions of interest

In [ ]:
st.slice_sequences(positions=[5, 10, 15])

Next consider a situation in which we create a site saturated library at 15 positions. In this example, the library size is much too large to have sampled every possible combination of bases. However a more likely situtation is one where we can easily observe all possible combinations of ** *3 positions* within the 15 available positions**

In other words, we know that we can easily observe how the distribution of individual positions within the 15 where biased and selected for in the library (that is ** *1 position within the 15* **), but it would also be useful to detect *linked mutations* or situations where a specific set of 3 bases were more biased than others

This is where the function *get_substrings* comes in. Rather than consider each base position independently, it will calculate the occurrence of each unique substring possible within the sequences provided. The total number of possible substrings can be calculated as follows:

* let n = total # of letters in sequence
* let k = substring length desired
* let a = number of unique letters in a sequence (A, C, T, G)

(n choose k) * a^k

From the example  above, lets slice 5 positions and then analyze all possible combinations of substrings of length 3

In [ ]:
# create a new instance of seqtable with just the 5 positions of interest
st_subset = seq_tables.seqtable(st.slice_sequences([2, 5, 10, 15, 16])['seqs'], encode_letters=False)
st_subset

In [ ]:
# calculate the distribution of all substrings of length 3 within this set of sequences whose length is 5
substring_counts = st_subset.get_substrings(word_length=3)
substring_counts

In the function above we asked to get all substrings within the seqtable of length 3. The result is a dataframe whose 
* columns represent all possible combinations of NCHOOSEK positions where N is the length of the sequence and K is the length of the substring (i.e. (1, 2, 3) represents slice all sequences at base position 1, 2, and 3). 
* rows represent all unique possible combinations of strings with length K (i.e. 4^k if letters are only [ACTG])
* the values represents the # of times a substring of sequences at defined positions (columns) contain the substring defined by the index/rows


In [ ]:
iplot(go.Figure(
 data=[go.Heatmap(
    x = [str(s) for s in substring_counts.columns],
    y = substring_counts.index,
    z = substring_counts.values
    )
    ],
layout=go.Layout(
        title='Heatmap of substring counts at defined positions',
        xaxis=dict(title='Set of positions'),
        yaxis=dict(title='substring')
)
))

** The result above shows that any set of positions containing (2, 3, 4) which are site-saturated, show an even distrubtion of all substrings at each position, 
where as substrings containing only one of the positions of site saturation (i.e. 1, 2, 5) show a much higher distribution of substrings with wildtype values at positions 1 and 5**